In [2]:
import postgres_connection as pc
import pandas as pd
import sqlite3

conn_info = {}
conn_info['host_name'] = 'tsdc-server'
conn_info['port_name'] = 5432
conn_info['db_name'] = 'master'
conn_info['user_name'] = 'jholden'
conn_info['password_name'] = 'NRELisgr8!'

# Get unique sampno, vehno

In [3]:
uni_id_quer = """
SELECT DISTINCT sampno, vehno
FROM tsdc_cmap.v_gpstrips_nrel
LIMIT 25
"""
uni_vehs_df = pc.query_table_df(uni_id_quer, conn_info)

# Get all trips for sample of vehicles

In [33]:
trips_quer = """
SELECT DISTINCT sampno, vehno, day_id, trip_id, start_ts, end_ts
FROM tsdc_cmap.v_gpstrips_nrel
WHERE (sampno, vehno)
IN """ + str(tuple([tuple(x) for x in uni_vehs_df.values]))

trips_df = pc.query_table_df(trips_quer, conn_info)

# Get all points for the sample of vehicles

In [11]:
quer = """
SELECT *
FROM tsdc_cmap.v_points_nrel
WHERE (sampno, vehno)
IN """ + str(tuple([tuple(x) for x in uni_vehs_df.values]))

# for i in range(len(uni_vehs_df)):
#     quer += "sampno = "+str(uni_vehs_df.sampno.iloc[i])+" AND vehno = "+str(uni_vehs_df.vehno.iloc[i])+" OR "
# quer = quer[:-3]

pnts_df = pc.query_table_df(quer, conn_info)

# Join trip identifiers to points data

Arbitrary unique trip ids are appending as an integer

In [34]:
trips_df['nrel_trip_id'] = range(len(trips_df))

In [41]:
pnts_df['nrel_trip_id'] = [-1]*len(pnts_df)

for i in range(len(trips_df)):
    row = trips_df.iloc[i]
    pnts_df.loc[(pnts_df['time_local']>=row['start_ts']) & (pnts_df['time_local']<=row['end_ts']), 'nrel_trip_id'] = row['nrel_trip_id']

Create SQLite file to write points data

In [52]:
pnts_cleansed = pnts_df[['sampno','vehno', 'nrel_trip_id','time_local','time_rel','gpsspeed']]

conn = sqlite3.connect('../cycles/cmap_subset/cmap_25_vehs.db')
pnts_cleansed.to_sql('gps_points', conn, if_exists='replace')
conn.close()